In [ ]:
%reset


In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime as dt
from matplotlib import pyplot as plt
from scipy import signal
from scipy.signal import butter, lfilter, freqz,filtfilt, welch,find_peaks
from scipy.fft import fft 
from scipy.special import entr
from scipy.stats import kurtosis,skew,entropy
from scipy.stats.stats import pearsonr 
from scipy.fftpack import fft
import math
import time
import seaborn as sns
import pickle
from statistics import stdev
from statistics import variance
import random
from detecta import detect_peaks
import pywt 

# Train data loading


In [ ]:

df = pd.read_csv("acc_user2.csv")
df = df.drop(['Unnamed: 0'],axis=1)
print('\n\nHead:\n\n',df.head())
print('\n\nTail:\n\n',df.tail())

print('\n\nShape: ' , df.shape)


# Checking Null and duplicate value and clean them

In [ ]:
print('\n\nNumber of null values :' ,df.isna().sum())
print('\n\nNumber of duplicate data :',df.duplicated().sum())

In [ ]:
df.dropna(inplace=True)
dfnew=df.drop_duplicates(keep='first')
print('\n\nAfter cleaning null data and duplicate data shape is :',dfnew.shape)

# Converting timezone into UTC

In [ ]:
dfnew.datetime = pd.to_datetime(dfnew.datetime)
dfnew.set_index('datetime', inplace =True)
dfnew = dfnew.tz_convert(tz='UTC')
dfnew.reset_index(inplace=True)
user = dfnew.copy()
user_id = input('Give the User_id no : ')

user['User_id'] = int(user_id)
print('\n\nAfter changing the timzone to UTC and adding the User_id column: ')
print('\nHead :\n\n',user.head())
print('Tail :\n\n',user.tail())
print('\n Shape : \n', user.shape)


In [ ]:
user=user.sort_values(by='datetime').reset_index(drop=True)
#df.set_index('datetime',inplace=True)
print('\n\nAfter sorting by the \'datetime\' column :')
print('\nHead :\n\n',user.head())
print('\nTail :\n\n',user.tail())
print('\n\n Shape:',user.shape)

# Loading and preocessing label data

In [ ]:
label= pd.read_csv('label_train.csv')
label.start = pd.to_datetime(label.start)
label.set_index('start', inplace =True)
label = label.tz_convert(tz='UTC')
label.reset_index(inplace=True)
label.finish = pd.to_datetime(label.finish)
label.set_index('finish', inplace =True)
label = label.tz_convert(tz='UTC')
label.reset_index(inplace=True)
print('\n \n Label data before cleaning :\n\n')
print('Label Head: \n\n\n',label.head())
print('Label Tail: \n\n\n',label.tail())
print('\n\n Shape:', label.shape)
label = label.drop(['id','role','activity_type','date','target_id','activity2user_id','target_role'],axis=1)
label.dropna(inplace=True)
label = label.drop_duplicates()


label1 = label[label.start==label.finish]
label2 = label[label.start != label.finish]
nots = dt.timedelta(seconds=59) 
label1.finish = label.finish + nots
label = label1.append(label2, ignore_index=True, sort=True)

label = label.reset_index(drop=True)
label = label.sort_values(by='start')
label = label.reset_index(drop = True)

label['finish']=pd.to_datetime(label['finish'])
label['start']=pd.to_datetime(label['start'])
print('\n\n Label data after cleaning : \n\n')
print('Label Head: \n\n\n',label.head())
print('Label Tail: \n\n\n',label.tail())
print('\n Shape:', label.shape)

In [ ]:
label.activity_type_id.value_counts()

# Checking the class imbalance in label data

In [ ]:
activity_map = {
1: "Vital", 
2: "Meal / medication", 
3: "Oral care", 
4: "Excretion", 
5: "Bathing / wiping",
6: "Treatment", 
7: "Morning gathering / exercises", 
8: "Rehabilitation / recreation", 
9: "Morning care", 
10: "Daytime user response", 
11: "Night care", 
12: "Nighttime user response", 
13: "Family / guest response", 
14: "Outing response", 
19: "Get up assistance", 
20: "Change dressing assistance", 
21: "Washing assistance", 
27: "Emergency response such as accident",
15: "Linen exchange", 
16: "Cleaning",
23: "Preparation and checking of goods", 
24: "Organization of medications",
17: "Handwriting recording", 
18: "Delegating / meeting", 
22: "Doctor visit correspondence", 
25: "Family / doctor contact",
26: "Break", 
28: "Special remarks / notes"
}

label["activity_type_e"]=label["activity_type_id"].map(activity_map)
label['activity_type_e'].value_counts().plot.bar()

# Checking where difference of timestamp is greater than 2 mins

In [ ]:
user=user.sort_values(by='datetime').reset_index(drop=True)
com = dt.timedelta(seconds=120)
user['datetime']=pd.to_datetime(user['datetime'])
d = user['datetime'].diff(periods=1)
breaki = user[(user['datetime'].diff(periods=1)>=com)] 
breaki=breaki.sort_values(by='datetime')
print('\n\nData that have \'timestamp\' difference with the previous data more than the desire value :  \n\n')
print(breaki.head())
print(breaki.tail())
print('\n\nShape :',breaki.shape)

# Creating new DF where timestamp difference is larger than 2 mins 

In [ ]:
d ={}
new=0
count=0
for i in tqdm(breaki.index):
    print (i)
    d[count] = pd.DataFrame()
    d[count]=user.loc[new:i-1]
    new = i
    count+=1
    if (i==breaki.index[-1]):
        d[count] = user.loc[new:]
    

In [ ]:
for i in tqdm(range(len(d))):
    print('d',i,'\n')
    print('head:',d[i].head())
    print('tail:',d[i].tail())
    print('Shape:',d[i].shape,'\n')


# Checking how many rows each new DF holds 

In [ ]:
#d3 = pd.Timestamp('2018-06-16 04:46:01.849000+00:00')
#delt = time.mktime(d3.timetuple())
#print(delt)
vlo =0
kharap=0
#print(len(d))
for i in tqdm(range(len(d))):
    if (len(d[i])<=5):
        vlo+=1
        if (d[i]["datetime"].nunique()==1):
            kharap+=1
        else:
            continue
    else : 
        continue
print('Number of DataFrame with only one data :', kharap)
print('Number of DataFrame with less or equal 5 data :', vlo)

# Resampling

In [ ]:

#unsampled1 = user1.resample('0.05S')
#inter1 = unsampled1.interpolate(method = 'spline',order=2)
#print(inter1.head())




#####x=df3['x'].to_numpy
delt = {}
f = {}
frame=pd.DataFrame()

if breaki.empty:
    print('\n\n This user has no data with datetime differance larger than that we desired\n\n ')
    lopa = pd.Timestamp(user['datetime'].iloc[-1])- pd.Timestamp(user["datetime"].iloc[0])
    lopa = lopa.total_seconds()
    lopa = (lopa*20)+1
    lopa = math.ceil(lopa)
    print('\n\n Number of total data in section {} is {}'. format(1,lopa))
    user['datetime']=pd.to_datetime(user['datetime'])
    frame['x']= signal.resample(user['x'],lopa)
    frame['y']=signal.resample(user['y'],lopa)
    frame['z']=signal.resample(user['z'],lopa)
    frame['datetime'] = pd.date_range(start=pd.Timestamp(user["datetime"].iloc[0]),periods=lopa ,freq='0.05S')
    print('\n\n Number of row of datetime of section {0} is {1} '.format(1,frame.datetime.nunique()))
    print('\nSection : 1\n')
    print('\n\n Head : \n\n',frame.head())
    print('\n\n Tail : \n\n',frame.tail())
    print('\n Shape:',frame.shape)
else :
    
    for i in tqdm(range(len(d))):
        
        if (pd.Timestamp(d[i]['datetime'].iloc[-1]) != pd.Timestamp(d[i]["datetime"].iloc[0])):
        
            delt[i] = pd.Timestamp(d[i]['datetime'].iloc[-1])- pd.Timestamp(d[i]["datetime"].iloc[0])
            delt[i] = delt[i].total_seconds()
            delt[i] = (delt[i]*20)+1
            delt[i]=math.ceil(delt[i])
            print('\n\n Number of total data in section {} is {}'. format(i,delt[i]))
            f[i] = pd.DataFrame()
            d[i]['datetime']=pd.to_datetime(d[i]['datetime'])
        
            #(f[i]['x'],f[i]['datetime'])= signal.resample(d[i]['x'],delt[i],d[i]['datetime'])
            f[i]['x']= signal.resample(d[i]['x'],delt[i])
            f[i]['y']=signal.resample(d[i]['y'],delt[i])
            f[i]['z']=signal.resample(d[i]['z'],delt[i])
            #f[i]['User_id'] = d[i]['User_id']
            f[i]['datetime'] = pd.date_range(start=pd.Timestamp(d[i]["datetime"].iloc[0]),periods=delt[i] ,freq='0.05S')
            print('\n\n Number of row of datetime of section {0} is {1} '.format(i,f[i].datetime.nunique()))
            print('\nSection', i ,'\n')
            print('\n\n Head : \n\n',f[i].head())
            print('\n\n Tail : \n\n',f[i].tail())
            print('\n Shape:',f[i].shape)
        
        elif (pd.Timestamp(d[i]['datetime'].iloc[-1]) == pd.Timestamp(d[i]["datetime"].iloc[0])):
            #f[i] = d[i]
            print('\n Here is  a single row')
            continue
            #print('\nSection', i+1 ,'\n')
            #print(f[i].head())
            #print(f[i].tail())
            #print('\n Shape:',f[i].shape)
        frame = frame.append([f[i]], ignore_index=False, sort=True)        
id_no = input('Give the User_id no : ')
frame['User_id'] = int(id_no)
frame.dropna(inplace=True)
frame=frame.drop_duplicates()
frame=frame.sort_values(by='datetime').reset_index(drop=True)
frame.datetime = pd.to_datetime(frame.datetime)
print('\nAfter concatanation :\n Head:\n \n \n ',frame.head())
print('\nTail:\n\n\n',frame.tail())
print('\nShape: ',frame.shape)

# Plotting the user data after and before resampling

In [ ]:
time = np.linspace(0,frame.index[-1]+1,frame.index[-1]+1)
plt.rcParams['figure.figsize']=(20,10)
plt.figure()
#plt.subplot(1,3,1)
plt.plot(time,frame.iloc[0:(frame.index[-1]+1)].x.values,'r.-',markersize=5,label='X')
#ax = plt.gca()
#ax.set_ylim([-12,12])
#ax.set_xlim([0,100])
#plt.subplot(1,3,2)
##plt.plot(time,dff22.iloc[20000:21000].y.values,'b.-',markersize=5,label='Y')
#ax = plt.gca()
#ax.set_ylim([-12,12])
#ax.set_xlim([0,100])
#plt.subplot(1,3,3)
##plt.plot(time,dff22.iloc[20000:21000].z.values,'g.-',markersize=5,label='Z')
plt.legend(loc='best')
ax = plt.gca()
ax.set_ylim([-30,30])
ax.set_xlim([0,frame.index[-1]+1])
plt.show()

In [ ]:
time = np.linspace(0,user.index[-1]+1,user.index[-1]+1)
plt.rcParams['figure.figsize']=(20,10)
plt.figure()
#plt.subplot(1,3,1)
plt.plot(time,user.iloc[0:user.index[-1]+1].x.values,'r.-',markersize=5,label='X')
#ax = plt.gca()
#ax.set_ylim([-12,12])
#ax.set_xlim([0,100])
#plt.subplot(1,3,2)
##plt.plot(time,dff22.iloc[20000:21000].y.values,'b.-',markersize=5,label='Y')
#ax = plt.gca()
#ax.set_ylim([-12,12])
#ax.set_xlim([0,100])
#plt.subplot(1,3,3)
##plt.plot(time,dff22.iloc[20000:21000].z.values,'g.-',markersize=5,label='Z')
plt.legend(loc='best')
ax = plt.gca()
ax.set_ylim([-25,25])
ax.set_xlim([0,user.index[-1]+1])
plt.show()

# Mapping the user data with label data

In [ ]:
frame.datetime = pd.to_datetime(frame.datetime)
data_new = pd.DataFrame()
take_count = 0

for i in tqdm(range(len(label))):
    mask = (label.loc[i,'start'] <= frame.datetime) & (frame.datetime <= label.loc[i,'finish']) & (frame.User_id == label.loc[i,'user_id'])
    part = frame.loc[mask].assign(take=take_count, label=label.loc[i,'activity_type_id'])
    data_new = data_new.append(part,ignore_index=True)
    if (part.empty == False):
        take_count = take_count + 1

In [ ]:
data_new.head()

In [ ]:
data_new.shape

In [ ]:
data_new.label.value_counts()

# Saving the labelled csv file and finally concating all the labelled csv file 

In [ ]:
data_new.to_csv('data5.csv',index = False)

In [ ]:
check={}
check[0] = pd.read_csv('data2.csv')
check[0].datetime = pd.to_datetime(check[0].datetime)
check[1] = pd.read_csv('data3.csv')
check[1].datetime = pd.to_datetime(check[1].datetime)
check[2] = pd.read_csv('data4.csv')
check[2].datetime = pd.to_datetime(check[2].datetime)
check[3] = pd.read_csv('data5.csv')
check[3].datetime = pd.to_datetime(check[3].datetime)
check[4] = pd.read_csv('data6.csv')
check[4].datetime = pd.to_datetime(check[4].datetime)
check[5] = pd.read_csv('data7.csv')
check[5].datetime = pd.to_datetime(check[5].datetime)
check[6] = pd.read_csv('data9.csv')
check[6].datetime = pd.to_datetime(check[6].datetime)
check[7] = pd.read_csv('data12.csv')
check[7].datetime = pd.to_datetime(check[7].datetime)
check[8] = pd.read_csv('data17.csv')
check[8].datetime = pd.to_datetime(check[8].datetime)
check[9] = pd.read_csv('data19.csv')
check[9].datetime = pd.to_datetime(check[9].datetime)
check[10] = pd.read_csv('data21.csv')
check[10].datetime = pd.to_datetime(check[10].datetime)
check[11] = pd.read_csv('data22.csv')
check[11].datetime = pd.to_datetime(check[11].datetime)


In [ ]:
data=pd.DataFrame()
for i in tqdm(range(len(check))):
    data = data.append([check[i]], ignore_index=True, sort=True)